<img src="https://heig-vd.ch/docs/default-source/doc-global-newsletter/2020-slim.svg" alt="HEIG-VD Logo" width="100" align="right" />

# Cours TAL – Labo 5 : Le modèle word2vec et ses applications

**Objectifs**
Le but de ce labo est de comparer un modèle word2vec pré-entraîné avec deux modèles que vous
entraînerez vous-mêmes, sur deux corpus de tailles différentes. La comparaison se fera sur une
tâche de similarité mots et sur une tâche de raisonnement par analogie, en anglais. Vous utiliserez la librairie Gensim de calcul de similarités pour le TAL.

## 1. Tester et évaluer un modèle déjà entraîné sur Google News

Installez gensim, une librairie Python qui fournit des outils pour travailler avec Word2Vec (avec
conda ou avec pip). **Attention** : la dernière version 4.2.3 de gensim est incompatible avec la
librairie scipy version 1.13, donc il faut installer la version 1.12 de scipy ; la variable Path doit
contenir `C:\ProgramData\Miniconda3\Library\` et `C:\ProgramData\Miniconda3\Library\bin\.`


In [ ]:
!pip install gensim

Obtenez depuis gensim le modèle word2vec pré-entraîné sur le corpus Google News en
écrivant : `w2v_vectors = gensim.downloader.load("word2vec-google-news-300")`, ce qui
téléchargera le fichier la première fois

In [3]:
import gensim.downloader

#Default path is C:\Users\username\gensim-data
w2v_vectors = gensim.downloader.load("word2vec-google-news-300")

[==================================================] 100.0% 1662.8/1662.8MB downloaded


Après avoir téléchargé le modèle, vous pouvez utiliser ainsi votre copie locale :
`w2v_vectors = KeyedVectors.load_word2vec_format(path_to_file, binary=True)`.

In [1]:
from gensim.models import KeyedVectors

w2v_vectors = KeyedVectors.load_word2vec_format("./corpus/GoogleNews-vectors-negative300.bin", binary=True)

#### a. Quelle place en mémoire occupe le processus du notebook avec les vecteurs de mots ?

Nous installant l'extension jupyter-server-resource-usage, nous avons pu observer que le kernel du notebook occupait 2.8 Go de mémoire vive après le chargement du modèle téléchargé en amont

![](./img/memory_usage.png)


#### b. Quelle est la dimension de l'espace vectoriel dans lequel les mots sont représentés ?



In [2]:
print(w2v_vectors.key_to_index.__len__()) #Nombre de clés = nombre de mots
print(w2v_vectors.vector_size) #Taille du vecteur pour chaque clé

3000000
300


Un `KeyedVector` est une structure semblable à un dictionnaire ayant comme clé un mot et comme valeur un vecteur. Dans notre cas, nous avons 3000000 clés chacune ayant un vecteur de 300 entrées.

#### c. Quelle est la taille du vocabulaire connu du modèle ? Veuillez afficher 5 mots anglais qui sont dans le vocabulaire et deux qui ne le sont pas.

In [3]:
voc_model = set(w2v_vectors.index_to_key)
result_in_voc = {"hello", "world", "computer", "science", "data"}.intersection(voc_model)
result_not_in_voc = {"crapulous", "manichaean"}.difference(voc_model)

print(f"Mots dans le vocabulaire {result_in_voc}") #5 mots
print(f"Mots pas dans le vocabulaire {result_not_in_voc}")
print(f"taille du vocabulaire {len(voc_model)}")

Mots dans le vocabulaire {'computer', 'science', 'hello', 'data', 'world'}
Mots pas dans le vocabulaire {'crapulous', 'manichaean'}
taille du vocabulaire 3000000


#### d. Quelle est la distance entre les mots rabbit et carrot ? Veuillez expliquer en une phrase comment on mesure les distances entre deux mots grâce à leurs vecteurs

In [4]:
print(f"Distance entre les mots: ", w2v_vectors.distance("rabbit", "carrot"))

Distance entre les mots:  0.6369356513023376


La distance entre deux mots est mesurée par la similarité cosinus entre les vecteurs de ces mots. Plus la valeur est proche de 0, plus les mots sont similaires, plus la valeur est proche de 1, plus les mots sont différents.

#### e. Considérez au moins 5 paires de mots anglais, certains proches par leurs sens, d’autres plus éloignés. Pour chaque paire, calculez la distance entre les deux mots. Veuillez indiquer si les distances obtenues correspondent à vos intuitions sur la proximité des sens des mots.

In [5]:
pairs = [("cat", "dog"), ("cat", "car"), ("hot", "cold"), ("shoe", "journalist"), ("height", "high")]

for pair in pairs:
    print(f"Distance entre les mots {pair}: ", w2v_vectors.distance(pair[0], pair[1]))
    

Distance entre les mots ('cat', 'dog'):  0.23905426263809204
Distance entre les mots ('cat', 'car'):  0.7847181558609009
Distance entre les mots ('hot', 'cold'):  0.5397861003875732
Distance entre les mots ('shoe', 'journalist'):  0.8940958231687546
Distance entre les mots ('height', 'high'):  0.8072148114442825


- **('cat', 'dog'):** Il s'agit de deux animaux de compagnie souvent mis en comparaison ou en opposition. Le score étant faible, il est cohérent avec notre intuition.
- **('cat', 'car'):** Ces deux mots sont proches au sens de l'orthographe, mais n'ont pas forcément un lien sémantique fort. On pourrait éventuellement supposer une distance faible si le système prenait en compte les erreurs typographiques, étant donné que T et R sont à côté l'un de l'autre sur un clavier QWERTY, mais ce n'est pas le cas. Le score étant élevé, il est cohérent avec notre intuition.
- **('hot', 'cold'):** Il s'agit de deux antonymes, ils possèdent donc un même sens sémantique (la température), mais leur nature d'antonymes pourrait également les éloigner. Le score est relativement moyen (~0.5) ce qui confirme notre intuition
- **('shoe', 'journalist'):** Ces deux mots n'ont rien en commun, on s'attend à un score élevé, ce qui est le cas.
- **('height', 'high'):** Ces deux mots sont similaires (l'un étant l'adjectif de l'autre), on s'attendrait à un score proche, voir moyen, mais le score est beaucoup plus élevé que notre intuition

#### f. Pouvez-vous trouver des mots de sens opposés mais qui sont proches selon le modèle ? Comment expliquez-vous cela ? Est-ce une qualité ou un défaut du modèle word2vec ?

In [6]:
print(f"Top 10 des mots les plus similaire: ", w2v_vectors.most_similar("good", topn=10))
print(f"Distance entre 'good' et 'bad': ", w2v_vectors.distance("good", "bad"))

Top 10 des mots les plus similaire:  [('great', 0.7291510105133057), ('bad', 0.7190051078796387), ('terrific', 0.6889115571975708), ('decent', 0.6837348341941833), ('nice', 0.6836092472076416), ('excellent', 0.644292950630188), ('fantastic', 0.6407778263092041), ('better', 0.6120728850364685), ('solid', 0.5806034803390503), ('lousy', 0.576420247554779)]
Distance entre 'good' et 'bad':  0.28099489212036133


Le mot "good" est proche de "bad" dans le modèle word2vec, ce qui est surprenant car ces deux mots sont des antonymes. Cela peut s'expliquer par le fait que les mots "good" et "bad" sont souvent utilisés dans des contextes similaires, par exemple dans des critiques de films ou de restaurants. C'est un défaut du modèle word2vec, car il ne prend pas en compte le sens des mots, mais seulement leur fréquence d'apparition dans un corpus de texte.

#### g. calculez le score du modèle word2vec sur les données WordSimilarity-353. Expliquez en 1-2 phrases comment ce score est calculé et ce qu’il mesure.

In [7]:
from gensim.test.utils import datapath


pearson, spearman, oov_ration = w2v_vectors.evaluate_word_pairs(datapath('wordsim353.tsv'))
print(pearson)
print(spearman)
print(oov_ration)

PearsonRResult(statistic=0.6238773472434951, pvalue=1.7963233960134136e-39)
SignificanceResult(statistic=0.6589215888009288, pvalue=2.5346056459149263e-45)
0.0


Le score de Pearson et le score de Spearman mesurent la corrélation entre les similarités de mots prédites par le modèle et les similarités de mots humaines (score de référence).
Le score de Pearson est une mesure de la corrélation linéaire entre deux variables, tandis que le score de Spearman est une mesure de la corrélation monotone. Un score élevé indique que le modèle prédit bien les similarités de mots humaines.
La pvalue est une valeur indiquant si le coefficient statistique est calculé par hasard. Plus cette valeur est faible, plus la corrélation statistique est significative (c.à.d. que la corrélation n'est pas due au hasard).

#### h. calculez le score du modèle word2vec sur les données questions-words.txt. Expliquez en 1-2 phrases comment ce score est calculé et ce qu’il mesure.

In [8]:
score, section = w2v_vectors.evaluate_word_analogies(datapath('questions-words.txt'), dummy4unknown=True)
print(score)

0.7320405239459681


On évalue ici l'analogie du modèle par rapport à certains mots dans certaines catégories de la façon suivante "a est à b ce que c est à d". Par exemple: "Athène est à la Grèce ce que Paris est à la France". Le modèle doit donc trouver le mot manquant d dans la phrase. Le score est calculé en fonction du nombre de réponses correctes trouvées par le modèle.